In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
import os
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision import models, datasets
from torch import nn, optim
from torch.cuda.amp import GradScaler, autocast
from sklearn.model_selection import train_test_split
import numpy as np
import time


In [16]:
# Parameters
base_dir = '/content/drive/MyDrive/JPEGImages'
batch_size = 64
num_workers = 8
pin_memory = True if torch.cuda.is_available() else False
num_epochs = 10
test_size = 0.2
random_seed = 42

# Transformations
transformations = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(30),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the dataset
dataset = datasets.ImageFolder(base_dir, transform=transformations)

In [17]:
# Splitting the dataset into train and validation sets
total_count = len(dataset)
test_count = int(total_count * test_size)
train_count = total_count - test_count
train_dataset, test_dataset = random_split(dataset, [train_count, test_count])

# Create data loaders with increased num_workers and pin_memory
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=pin_memory)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=pin_memory)

In [18]:
# Model setup
model = models.resnet50(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 50)  # Adjust for the number of classes
device='cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
scaler = GradScaler()


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [19]:
# Training and evaluation functions as defined previously

# Training function
def train_and_evaluate(model, train_loader, test_loader, criterion, optimizer, scheduler, scaler, num_epochs):
    start_time = time.time()  # Start time
    model.train()
    for epoch in range(num_epochs):
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)

            with autocast():
                outputs = model(inputs)
                loss = criterion(outputs, labels)

            optimizer.zero_grad()
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            if (i + 1) % 20 == 0:
                print(f'Epoch {epoch+1}, Step {i+1}, Loss: {loss.item():.4f}')

        scheduler.step()

    end_time = time.time()  # End time
    total_time = end_time - start_time
    print(f'Total training time is {total_time:.2f} seconds')

    # Evaluation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy on test set: {accuracy}%')




In [20]:
# Run training and evaluation
train_and_evaluate(model, train_loader, test_loader, criterion, optimizer, scheduler, scaler, num_epochs)

Epoch 1, Step 20, Loss: 2.4880
Epoch 1, Step 40, Loss: 2.5202
Epoch 1, Step 60, Loss: 1.8933
Epoch 1, Step 80, Loss: 1.7236
Epoch 1, Step 100, Loss: 1.9150
Epoch 1, Step 120, Loss: 1.6164
Epoch 2, Step 20, Loss: 1.6798
Epoch 2, Step 40, Loss: 1.6901
Epoch 2, Step 60, Loss: 1.7322
Epoch 2, Step 80, Loss: 1.1315
Epoch 2, Step 100, Loss: 1.4017
Epoch 2, Step 120, Loss: 1.2431
Epoch 3, Step 20, Loss: 1.2470
Epoch 3, Step 40, Loss: 1.4630
Epoch 3, Step 60, Loss: 1.3431
Epoch 3, Step 80, Loss: 1.0082
Epoch 3, Step 100, Loss: 1.2643
Epoch 3, Step 120, Loss: 1.1583
Epoch 4, Step 20, Loss: 0.7844
Epoch 4, Step 40, Loss: 1.0242
Epoch 4, Step 60, Loss: 0.8186
Epoch 4, Step 80, Loss: 1.1498
Epoch 4, Step 100, Loss: 1.1314
Epoch 4, Step 120, Loss: 0.9672
Epoch 5, Step 20, Loss: 0.7974
Epoch 5, Step 40, Loss: 1.0652
Epoch 5, Step 60, Loss: 0.9407
Epoch 5, Step 80, Loss: 0.9256
Epoch 5, Step 100, Loss: 0.7910
Epoch 5, Step 120, Loss: 1.3149
Epoch 6, Step 20, Loss: 0.8113
Epoch 6, Step 40, Loss: 0.596